In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    HPFlexSurvey,
    convert_units,
    SHACLHandler
)
from buildingmotif.namespaces import BRICK, RDF
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey('test_site','test_build','.', overwrite=True)

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values names: {}
values: dict_values(['area-name-value'])
values names: {'area-name': '<name>-area-name'}
values: dict_values(['heating_COP-name-value', 'cooling_capacity-name-value', 'cooling_COP-name-value', 'heating_capacity-name-value'])
values names: {'cooling_COP-name': '<name>-cooling_COP-name', 'heating_COP-name': '<name>-heating_COP-name', 'cooling_capacity-name': '<name>-cooling_capacity-name', 'heating_capacity-name': '<name>-heating_capacity-name'}
values: dict_values(['tolerance-name-value', 'active-name-value', 'resolution-name-value', 'setpoint_deadband-name-value', 'stage_count-name-value'])
values names: {'tolerance-name': '<name>-tolerance-name', 'resolution-name': '<name>-resolution-name', 'setpoint_deadband-name': '<name>-setpoint_deadband-name', 'active-name': '<name>-active-name', 'stage_count-name': '<name>-stage_count-name'}
v

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey('test_site/test_build')

Loading config from: test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 8 empty cells across 2 columns in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area-name']
Expanded CSV shape: (3, 9)
New columns added: ['cooling_COP-name', 'heating_COP-name', 'cooling_capacity-name', 'heating_capacity-name']
Expanded CSV shape: (3, 14)
New columns added: ['tolerance-name', 'resolution-name', 'setpoint_deadband-name', 'active-name', 'stage_count-name']
Expanded CSV shape: (7, 8)
New columns added: ['tilt-name', 'azimuth-name', 'area-name']
Expanded CSV shape: (1, 9)
New columns added: ['noaastation-name', 'timezone-name', 'longitude-name', 'latitude-name']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)

In [8]:
# Create handler
handler = SHACLHandler(ontology='brick')

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(s.graph)

s.graph.serialize('test-brick-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [9]:
# lots of new inferred information
(s.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/test_site#hvac_1> a ns1:hp-rtu ;
    ns1:air-connects-to <urn:hpflex/test_site#zone_1> .

<urn:hpflex/test_site#hvac_2> a ns1:hp-rtu ;
    ns1:air-connects-to <urn:hpflex/test_site#zone_2> .

<urn:hpflex/test_site#hvac_3> a ns1:hp-rtu ;
    ns1:air-connects-to <urn:hpflex/test_site#zone_3> .

<urn:hpflex/test_site#test_site> a ns1:site .

<urn:hpflex/test_site#tstat_zone_1> a ns1:tstat ;
    ns1:has-location <urn:hpflex/test_site#zone_1> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_1-active-name>,
        <urn:hpflex/test_site#tstat_zone_1-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_1-tolerance-name> .

<urn:hpflex/test_site#tstat_zone_2> a ns1:tstat ;
    ns1:has-location <urn:hpflex/test_site#zone_2> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_2-active-name>,
        <urn:hpflex/test_site#tstat_zone_2-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_2-tolerance-name> .

<urn:hpfle

# Testing get Metadata

In [10]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel("test_site/test_build/test_build.ttl", ontology = 'brick')
site_info = loader.get_all_building_objects()

In [11]:
print(site_info)

{'sites': [Site(name='urn:hpflex/test_site#test_site', longitudes=1, latitudes=1, noaastations=1, timezones=1)], 'zones': [Hvaczone(name='urn:hpflex/test_site#zone_3', tstats=1, hvacs=1, windows=1, spaces=1), Hvaczone(name='urn:hpflex/test_site#zone_2', tstats=1, hvacs=1, windows=1, spaces=1), Hvaczone(name='urn:hpflex/test_site#zone_1', tstats=1, hvacs=1, windows=2, spaces=2)]}


In [12]:
zone = site_info['zones'][0]
print(zone)

Hvaczone(name='urn:hpflex/test_site#zone_3', tstats=1, hvacs=1, windows=1, spaces=1)


In [13]:
zone.windows

[Window(name='urn:hpflex/test_site#window_3_3', azimuth=Value(value=7.0, unit='None'), area=Value(value=7.0, unit='http://qudt.org/vocab/unit/FT2'), tilt=Value(value=7.0, unit='None'))]

In [14]:
zone.windows[0].area

Value(value=7.0, unit='http://qudt.org/vocab/unit/FT2')

In [15]:
zone.tstats[0]

Tstat(name='urn:hpflex/test_site#tstat_zone_3', tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), active=Value(value=3.0, unit='None'), stage_count=Value(value=3.0, unit='None'), setpoint_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), resolution=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'))

In [16]:
print(zone.tstats[0].resolution)
zone.tstats[0].resolution.convert_to_si()
print(zone.tstats[0].resolution)

Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F')
CONVERT http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
Value(value=-16.111111111111086, unit='http://qudt.org/vocab/unit/DEG_C')


# Testing Unit Conversion

In [17]:
convert_units(10, 'FT', 'M')

3.048

In [18]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [19]:
convert_units(0, 'DEG_C', 'K')

273.15

In [20]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [21]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [22]:
# bg.create_dashboard('AQL from brick')

In [23]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')